# Validation RL-Element

### Import dpsimpy 

In [ ]:
import dpsimpy

### Parameters

In [ ]:
resistance = 10
inductance = 1e-3
capacitance = 5e-6
frequency = 50

### Powerflow for Initialization

In [ ]:
sim_name_pf = 'VS_RLC1_PF'
dpsimpy.Logger.set_log_dir("logs/" + sim_name_pf)

# Nodes
gnd_pf = dpsimpy.sp.SimNode.gnd
n1_pf  = dpsimpy.sp.SimNode('n1_pf')
n2_pf  = dpsimpy.sp.SimNode('n2_pf')
n3_pf  = dpsimpy.sp.SimNode('n3_pf')

# Components
vs_pf = dpsimpy.sp.ph1.VoltageSource('vs_pf', dpsimpy.LogLevel.debug)
vs_pf.set_parameters(V_ref=complex(10,0))
r1_pf = dpsimpy.sp.ph1.Resistor('r1_pf', dpsimpy.LogLevel.debug)
r1_pf.set_parameters(R=resistance)
c1_pf = dpsimpy.sp.ph1.Capacitor('c1_pf', dpsimpy.LogLevel.debug)
c1_pf.set_parameters(C=capacitance)
l1_pf = dpsimpy.sp.ph1.Inductor('l1_pf', dpsimpy.LogLevel.debug)
l1_pf.set_parameters(L=inductance)

# Connections
vs_pf.connect([gnd_pf, n1_pf])
r1_pf.connect([n2_pf, n1_pf])
l1_pf.connect([n3_pf, n2_pf])
c1_pf.connect([gnd_pf, n3_pf])

# Define system topology
system_pf = dpsimpy.SystemTopology(frequency, [n1_pf, n2_pf, n3_pf], [vs_pf, r1_pf, l1_pf, c1_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute('n1.v', 'v', n1_pf)
logger_pf.log_attribute('n2.v', 'v', n2_pf)
logger_pf.log_attribute('n3.v', 'v', n3_pf)
logger_pf.log_attribute('r1.i_intf', 'i_intf', r1_pf)
logger_pf.log_attribute('l1.i_intf', 'i_intf', l1_pf)

sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_time_step(0.1)
sim_pf.set_final_time(0.5)
sim_pf.add_logger(logger_pf)
sim_pf.run()

## 1. DP Domain

### RLC circuit (reference)

In [ ]:
name = 'DP_VS_RLC'
dpsimpy.Logger.set_log_dir("logs/" + name)

# Nodes
gnd = dpsimpy.dp.SimNode.gnd
n1 = dpsimpy.dp.SimNode('n1')
n1.set_initial_voltage(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0])
n2 = dpsimpy.dp.SimNode('n2')
n2.set_initial_voltage(sim_pf.get_idobj_attr(n2_pf.name(), 'v').get()[0])
n3 = dpsimpy.dp.SimNode('n3')
n3.set_initial_voltage(sim_pf.get_idobj_attr(n3_pf.name(), 'v').get()[0])

# Components
vs = dpsimpy.dp.ph1.VoltageSource('vs')
vs.set_parameters(V_ref=complex(10,0))
r1 = dpsimpy.dp.ph1.Resistor('r_1', dpsimpy.LogLevel.off)
r1.R = resistance
l1 = dpsimpy.dp.ph1.Inductor('l_1', dpsimpy.LogLevel.off)
l1.L = inductance
c1 = dpsimpy.dp.ph1.Capacitor('c1', dpsimpy.LogLevel.off)
c1.set_parameters(C=capacitance)

# Connections
vs.connect([gnd, n1])
r1.connect([n2, n1])
l1.connect([n3, n2])
c1.connect([gnd, n3])

# Define system topology
system = dpsimpy.SystemTopology(frequency, [gnd, n1, n2, n3], [vs, r1, l1, c1])

# Logging
logger = dpsimpy.Logger(name)
logger.log_attribute('n1.v', 'v', n1)
logger.log_attribute('n2.v', 'v', n2)
logger.log_attribute('n3.v', 'v', n3)
logger.log_attribute('l1.i_intf', 'i_intf', l1)


sim = dpsimpy.Simulation(name, dpsimpy.LogLevel.off)
sim.set_system(system)
sim.set_domain(dpsimpy.Domain.DP)
sim.set_time_step(0.0001)
sim.set_final_time(0.02)
sim.add_logger(logger)
sim.run()

### Read Results

In [ ]:
# read Simulink log file

import matplotlib.pyplot as plt
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import villas.dataprocessing.plottools as pt
import os

# read EMT results
file_path = os.getcwd() + "/logs/" + name + "/" + name + ".csv"
ts_dpsim_comps = rt.read_timeseries_dpsim(file_path)

#convert to emt
ts_dpsim_comps = ts.frequency_shift_list(ts_dpsim_comps, frequency)

### Circuit with RL-Element

In [ ]:
name_dp = 'DP_RL-element'
dpsimpy.Logger.set_log_dir("logs/" + name_dp)

# Nodes
gnd = dpsimpy.dp.SimNode.gnd
n1 = dpsimpy.dp.SimNode('n1')
n1.set_initial_voltage(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0])
n2 = dpsimpy.dp.SimNode('n2')
n2.set_initial_voltage(sim_pf.get_idobj_attr(n3_pf.name(), 'v').get()[0])

# Components
vs = dpsimpy.dp.ph1.VoltageSource('vs', dpsimpy.LogLevel.debug)
vs.set_parameters(V_ref=complex(10,0))
rl = dpsimpy.dp.ph1.ResInductor('rl', dpsimpy.LogLevel.debug)
rl.set_parameters(R=resistance, L=inductance)
c1 = dpsimpy.dp.ph1.Capacitor('c1', dpsimpy.LogLevel.debug)
c1.set_parameters(C=capacitance)

# Connections
vs.connect([gnd, n1])
rl.connect([n2, n1])
c1.connect([gnd, n2])

# Define system topology
system = dpsimpy.SystemTopology(frequency, [gnd, n1, n2], [vs, rl, c1])

# Logging
logger = dpsimpy.Logger(name_dp)
logger.log_attribute('n1.v', 'v', n1)
logger.log_attribute('n2.v', 'v', n2)
logger.log_attribute('rl.i_intf', 'i_intf', rl)
logger.log_attribute('rl.v', 'v_intf', rl)

sim = dpsimpy.Simulation(name_dp)
sim.set_system(system)
sim.set_domain(dpsimpy.Domain.DP)
sim.set_time_step(0.0001)
sim.set_final_time(0.02)
sim.add_logger(logger)
sim.run()

### Read Results

In [ ]:
# read DP results
file_path = os.getcwd() + "/logs/" + name_dp + "/" + name_dp + ".csv"
ts_dpsim_dp = rt.read_timeseries_dpsim(file_path)

#convert to emt
ts_dpsim_dp = ts.frequency_shift_list(ts_dpsim_dp, frequency)

### Plot results

In [ ]:
ts_dpsim_comps["n3.v_shift"].label = 'vc - RLC circuit'
ts_dpsim_comps["l1.i_intf_shift"].label = 'i - RLC circuit'

ts_dpsim_dp["n2.v_shift"].label = 'vc RL Element - DP'
ts_dpsim_dp["rl.i_intf_shift"].label = 'i RL Element - DP'

pt.plot_timeseries(1, ts_dpsim_comps["n3.v_shift"])
pt.plot_timeseries(1, ts_dpsim_dp["n2.v_shift"], plt_linestyle='--')

pt.plot_timeseries(2, ts_dpsim_comps["l1.i_intf_shift"])
pt.plot_timeseries(2, ts_dpsim_dp["rl.i_intf_shift"], plt_linestyle='--')

### Assert 

In [45]:
# calculate the RMSE between both models
err_current = ts.rmse(ts_dpsim_comps["l1.i_intf_shift"], ts_dpsim_dp["rl.i_intf_shift"])
err_voltage = ts.rmse(ts_dpsim_comps["n3.v_shift"], ts_dpsim_dp["n2.v_shift"])

assert err_current < 0.00001
assert err_voltage < 0.00001

## 2. EMT

### RLC Circuit (reference)

In [ ]:
name_emt_ref = 'EMT_RL-ref'
dpsimpy.Logger.set_log_dir("logs/" + name_emt_ref)

# Nodes
gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode('n1', dpsimpy.PhaseType.ABC)
n1.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0][0]))
n2 = dpsimpy.emt.SimNode('n2', dpsimpy.PhaseType.ABC)
n2.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(sim_pf.get_idobj_attr(n2_pf.name(), 'v').get()[0][0]))
n3 = dpsimpy.emt.SimNode('n3', dpsimpy.PhaseType.ABC)
n3.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(sim_pf.get_idobj_attr(n3_pf.name(), 'v').get()[0][0]))

# Components
vs = dpsimpy.emt.ph3.VoltageSource('vs', dpsimpy.LogLevel.off)
vs.set_parameters(V_ref=dpsimpy.Math.single_phase_variable_to_three_phase(complex(10, 0)), f_src=frequency)
r1 = dpsimpy.emt.ph3.Resistor('r1', dpsimpy.LogLevel.off)
r1.set_parameters(R=dpsimpy.Math.single_phase_parameter_to_three_phase(resistance))
l1 = dpsimpy.emt.ph3.Inductor('l1', dpsimpy.LogLevel.off)
l1.set_parameters(L=dpsimpy.Math.single_phase_parameter_to_three_phase(inductance))
c1 = dpsimpy.emt.ph3.Capacitor('c1', dpsimpy.LogLevel.off)
c1.set_parameters(C=dpsimpy.Math.single_phase_parameter_to_three_phase(capacitance))

# Connections
vs.connect([gnd, n1])
r1.connect([n2, n1])
l1.connect([n3, n2])
c1.connect([gnd, n3])


# Define system topology
system = dpsimpy.SystemTopology(frequency, [gnd, n1, n2, n3], [vs, r1, l1, c1])

# Logging
logger = dpsimpy.Logger(name_emt_ref)
logger.log_attribute('n1.v', 'v', n1)
logger.log_attribute('n2.v', 'v', n2)
logger.log_attribute('n3.v', 'v', n3)
logger.log_attribute('r1.i_intf', 'i_intf', r1)
logger.log_attribute('r1.v', 'v_intf', r1)
logger.log_attribute('l1.i_intf', 'i_intf', l1)
logger.log_attribute('l1.v', 'v_intf', l1)

sim = dpsimpy.Simulation(name_emt_ref, dpsimpy.LogLevel.off)
sim.set_system(system)
sim.set_domain(dpsimpy.Domain.EMT)
sim.set_time_step(0.0001)
sim.set_final_time(0.02)
sim.add_logger(logger)
sim.run()

### Read results

In [ ]:
# read DP results
file_path = os.getcwd() + "/logs/" + name_emt_ref + "/" + name_emt_ref + ".csv"
ts_dpsim_emt_ref = rt.read_timeseries_dpsim(file_path)

### Circuit with RL-Element

In [ ]:
name_emt = 'EMT_RL-element'
dpsimpy.Logger.set_log_dir("logs/" + name_emt)

# Nodes
gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode('n1', dpsimpy.PhaseType.ABC)
n1.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0][0]))
n2 = dpsimpy.emt.SimNode('n2', dpsimpy.PhaseType.ABC)
n2.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(sim_pf.get_idobj_attr(n3_pf.name(), 'v').get()[0][0]))

# Components
vs = dpsimpy.emt.ph3.VoltageSource('vs', dpsimpy.LogLevel.off)
vs.set_parameters(V_ref=dpsimpy.Math.single_phase_variable_to_three_phase(complex(10, 0)), f_src=frequency)
rl = dpsimpy.emt.ph3.ResInductor('rl', dpsimpy.LogLevel.off)
rl.set_parameters(R=dpsimpy.Math.single_phase_parameter_to_three_phase(resistance), L=dpsimpy.Math.single_phase_parameter_to_three_phase(inductance))
c1 = dpsimpy.emt.ph3.Capacitor('c1', dpsimpy.LogLevel.off)
c1.set_parameters(C=dpsimpy.Math.single_phase_parameter_to_three_phase(capacitance))

# Connections
vs.connect([gnd, n1])
rl.connect([n2, n1])
c1.connect([gnd, n2])

# Define system topology
system = dpsimpy.SystemTopology(frequency, [gnd, n1, n2], [vs, rl, c1])

# Logging
logger = dpsimpy.Logger(name_emt)
logger.log_attribute('n1.v', 'v', n1)
logger.log_attribute('n2.v', 'v', n2)
logger.log_attribute('rl.i_intf', 'i_intf', rl)
logger.log_attribute('rl.v', 'v_intf', rl)

sim = dpsimpy.Simulation(name_emt, dpsimpy.LogLevel.off)
sim.set_system(system)
sim.set_domain(dpsimpy.Domain.EMT)
sim.set_time_step(0.00001)
sim.set_final_time(0.02)
sim.add_logger(logger)
sim.run()

### Read EMT results

In [ ]:
# read DP results
file_path = os.getcwd() + "/logs/" + name_emt + "/" + name_emt + ".csv"
ts_dpsim_emt = rt.read_timeseries_dpsim(file_path)

### Plot Results

In [ ]:
ts_dpsim_emt_ref["l1.i_intf_0"].label = 'i - REF'
ts_dpsim_emt_ref["n3.v_0"].label = 'vc - REF'
ts_dpsim_emt["rl.i_intf_0"].label = 'i RL Element'
ts_dpsim_emt["n2.v_0"].label = 'vc RL Element'

pt.plot_timeseries(3, ts_dpsim_emt_ref["n3.v_0"])
pt.plot_timeseries(3, ts_dpsim_emt["n2.v_0"], plt_linestyle='--')

pt.plot_timeseries(4, ts_dpsim_emt_ref["l1.i_intf_0"])
pt.plot_timeseries(4, ts_dpsim_emt["rl.i_intf_0"], plt_linestyle='--')

### Assert

In [46]:
# calculate the RMSE between both models
err_current = ts.rmse(ts_dpsim_emt_ref["l1.i_intf_0"], ts_dpsim_emt["rl.i_intf_0"])
err_voltage = ts.rmse(ts_dpsim_emt_ref["n3.v_0"], ts_dpsim_emt["n2.v_0"])

assert err_current < 0.001
assert err_voltage < 0.001

## 3. SP

### RLC circuit (reference)

In [ ]:
name_sp_ref = 'SP_VS_RLC'
dpsimpy.Logger.set_log_dir("logs/" + name_sp_ref)

# Nodes
gnd = dpsimpy.sp.SimNode.gnd
n1 = dpsimpy.sp.SimNode('n1')
n1.set_initial_voltage(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0])
n2 = dpsimpy.sp.SimNode('n2')
n2.set_initial_voltage(sim_pf.get_idobj_attr(n2_pf.name(), 'v').get()[0])
n3 = dpsimpy.sp.SimNode('n3')
n3.set_initial_voltage(sim_pf.get_idobj_attr(n3_pf.name(), 'v').get()[0])

# Components
vs = dpsimpy.sp.ph1.VoltageSource('vs')
vs.set_parameters(V_ref=complex(10,0))
r1 = dpsimpy.sp.ph1.Resistor('r_1', dpsimpy.LogLevel.info)
r1.set_parameters(R=resistance)
l1 = dpsimpy.sp.ph1.Inductor('l_1', dpsimpy.LogLevel.info)
l1.set_parameters(L=inductance)
c1 = dpsimpy.sp.ph1.Capacitor('c1', dpsimpy.LogLevel.info)
c1.set_parameters(C=capacitance)

# Connections
vs.connect([gnd, n1])
r1.connect([n2, n1])
l1.connect([n3, n2])
c1.connect([gnd, n3])

# Define system topology
system = dpsimpy.SystemTopology(frequency, [gnd, n1, n2, n3], [vs, r1, l1, c1])

# Logging
logger = dpsimpy.Logger(name_sp_ref)
logger.log_attribute('n1.v', 'v', n1)
logger.log_attribute('n2.v', 'v', n2)
logger.log_attribute('n3.v', 'v', n3)
logger.log_attribute('l1.i_intf', 'i_intf', l1)


sim = dpsimpy.Simulation(name_sp_ref, dpsimpy.LogLevel.info)
sim.set_system(system)
sim.set_domain(dpsimpy.Domain.SP)
sim.set_time_step(0.0001)
sim.set_final_time(0.02)
sim.add_logger(logger)
sim.run()

### Read Results

In [ ]:
# read Simulink log file

# read EMT results
file_path = os.getcwd() + "/logs/" + name_sp_ref + "/" + name_sp_ref + ".csv"
ts_dpsim_sp_ref = rt.read_timeseries_dpsim(file_path)

#convert to emt
ts_dpsim_sp_ref = ts.frequency_shift_list(ts_dpsim_sp_ref, frequency)

### Circuit with RL-Element

In [ ]:
name_sp = 'SP_RL-element'
dpsimpy.Logger.set_log_dir("logs/" + name_sp)

# Nodes
gnd = dpsimpy.sp.SimNode.gnd
n1 = dpsimpy.sp.SimNode('n1')
n1.set_initial_voltage(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0])
n2 = dpsimpy.sp.SimNode('n2')
n2.set_initial_voltage(sim_pf.get_idobj_attr(n3_pf.name(), 'v').get()[0])

# Components
vs = dpsimpy.sp.ph1.VoltageSource('vs', dpsimpy.LogLevel.debug)
vs.set_parameters(V_ref=complex(10,0))
rl = dpsimpy.sp.ph1.ResInductor('rl', dpsimpy.LogLevel.debug)
rl.set_parameters(R=resistance, L=inductance)
c1 = dpsimpy.sp.ph1.Capacitor('c1', dpsimpy.LogLevel.debug)
c1.set_parameters(C=capacitance)

# Connections
vs.connect([gnd, n1])
rl.connect([n2, n1])
c1.connect([gnd, n2])

# Define system topology
system = dpsimpy.SystemTopology(frequency, [gnd, n1, n2], [vs, rl, c1])

# Logging
logger = dpsimpy.Logger(name_sp)
logger.log_attribute('n1.v', 'v', n1)
logger.log_attribute('n2.v', 'v', n2)
logger.log_attribute('rl.i_intf', 'i_intf', rl)
logger.log_attribute('rl.v', 'v_intf', rl)

sim = dpsimpy.Simulation(name_sp)
sim.set_system(system)
sim.set_domain(dpsimpy.Domain.SP)
sim.set_time_step(0.0001)
sim.set_final_time(0.02)
sim.add_logger(logger)
sim.run()

### Read Results

In [ ]:
# read Simulink log file

# read EMT results
file_path = os.getcwd() + "/logs/" + name_sp + "/" + name_sp + ".csv"
ts_dpsim_sp = rt.read_timeseries_dpsim(file_path)

#convert to emt
ts_dpsim_sp = ts.frequency_shift_list(ts_dpsim_sp, frequency)

### Plot Results

In [ ]:
ts_dpsim_sp_ref["l1.i_intf_shift"].label = 'i - REF'
ts_dpsim_sp_ref["n3.v_shift"].label = 'vc - REF'
ts_dpsim_sp["rl.i_intf_shift"].label = 'i RL Element'
ts_dpsim_sp["n2.v_shift"].label = 'vc RL Element'

pt.plot_timeseries(5, ts_dpsim_sp_ref["n3.v_shift"])
pt.plot_timeseries(5, ts_dpsim_sp["n2.v_shift"], plt_linestyle='--')

pt.plot_timeseries(6, ts_dpsim_sp_ref["l1.i_intf_shift"])
pt.plot_timeseries(6, ts_dpsim_sp["rl.i_intf_shift"], plt_linestyle='--')

### Assert

In [44]:
# calculate the RMSE between both models
err_current = ts.rmse(ts_dpsim_sp_ref["l1.i_intf_shift"], ts_dpsim_sp["rl.i_intf_shift"])
err_voltage = ts.rmse(ts_dpsim_sp_ref["n3.v_shift"], ts_dpsim_sp["n2.v_shift"])

assert err_current < 0.00001
assert err_voltage < 0.00001